(lightning_mnist_example)=

# Train a Pytorch Lightning Image Classifier

This example introduces how to train a Pytorch Lightning Module using AIR {class}`LightningTrainer <ray.train.lightning.LightningTrainer>`. We will demonstrate how to train a basic neural network on the MNIST dataset with distributed data parallelism.


In [49]:
SMOKE_TEST = True

In [50]:
import os
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from filelock import FileLock
from torch.utils.data import DataLoader, random_split, Subset
from torchmetrics import Accuracy
from torchvision.datasets import MNIST
from torchvision import transforms

import pytorch_lightning as pl
from pytorch_lightning import trainer
from pytorch_lightning.core import datamodule
from pytorch_lightning.loggers.csv_logs import CSVLogger

## Prepare Dataset and Module

The Pytorch Lightning Trainer takes either `torch.utils.data.DataLoader` or `pl.LightningDataModule` as data inputs. You can keep using them without any changes for the Ray AIR LightningTrainer. 

In [51]:
class MNISTDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=100):
        super().__init__()
        self.data_dir = os.getcwd()
        self.batch_size = batch_size
        self.transform = transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
        )

    def setup(self, stage=None):
        with FileLock(f"{self.data_dir}.lock"):
            mnist = MNIST(
                self.data_dir, train=True, download=True, transform=self.transform
            )

            # split data into train and val sets
            self.mnist_train, self.mnist_val = random_split(mnist, [55000, 5000])

    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=self.batch_size, num_workers=4)

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=self.batch_size, num_workers=4)

    def test_dataloader(self):
        with FileLock(f"{self.data_dir}.lock"):
            self.mnist_test = MNIST(
                self.data_dir, train=False, download=True, transform=self.transform
            )
        return DataLoader(self.mnist_test, batch_size=self.batch_size, num_workers=4)


datamodule = MNISTDataModule(batch_size=128)

Next, define a simple multi-layer perception as the subclass of `pl.LightningModule`.

In [52]:
class MNISTClassifier(pl.LightningModule):
    def __init__(self, lr=1e-3, feature_dim=128):
        torch.manual_seed(421)
        super(MNISTClassifier, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, feature_dim),
            nn.ReLU(),
            nn.Linear(feature_dim, 10),
            nn.ReLU(),
        )
        self.lr = lr
        self.accuracy = Accuracy()

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.linear_relu_stack(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = torch.nn.functional.cross_entropy(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, val_batch, batch_idx):
        loss, acc = self._shared_eval(val_batch)
        self.log("val_accuracy", acc)
        return {"val_loss": loss, "val_accuracy": acc}

    def test_step(self, test_batch, batch_idx):
        loss, acc = self._shared_eval(test_batch)
        self.log("test_accuracy", acc)
        return {"test_loss": loss, "test_accuracy": acc}

    def _shared_eval(self, batch):
        x, y = batch
        logits = self.forward(x)
        loss = F.nll_loss(logits, y)
        acc = self.accuracy(logits, y)
        return loss, acc

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        avg_acc = torch.stack([x["val_accuracy"] for x in outputs]).mean()
        self.log("val_loss", avg_loss, sync_dist=True)
        self.log("val_accuracy", avg_acc, sync_dist=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

You don't need to make any change to the definition of PyTorch Lightning model and datamodule.

## Define the Cofigurations for AIR LightningTrainer

The {meth}`LightningConfigBuilder <ray.train.lightning.LightningConfigBuilder>` class stores all the parameters involved in training a PyTorch Lightning module. It takes the same parameter lists as those in PyTorch Lightning.

- The `.module()` method takes a subclass of `pl.LightningModule` and its initialization parameters. `LightningTrainer` will instantiate a model instance internally in the workers' training loop.
- The `.trainer()` method takes the initialization parameters of `pl.Trainer`. You can specify training configurations, loggers, and callbacks here.
- The `.fit_params()` method stores all the parameters that will be passed into `pl.Trainer.fit()`, including train/val dataloaders, datamodules, and checkpoint paths.
- The `.checkpointing()` method saves the configurations for a `RayModelCheckpoint` callback. This callback reports the latest metrics to the AIR session along with a newly saved checkpoint.
- The `.build()` method generates a dictionary that contains all the configurations in the builder. This dictionary will be passed to `LightningTrainer` later.

Next, let's go step-by-step to see how to convert your existing PyTorch Lightning training script to a LightningTrainer.

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint
from ray.air.config import RunConfig, ScalingConfig, CheckpointConfig
from ray.train.lightning import (
    LightningTrainer,
    LightningConfigBuilder,
    LightningCheckpoint,
)


def build_lightning_config_from_existing_code():
    # Create a config builder to encapsulate all required parameters.
    # Note that model instantiation and fitting will occur later in the LightingTrainer,
    # rather than in the config builder.
    config_builder = LightningConfigBuilder()

    # 1. define your model
    # model = MNISTClassifier(lr=1e-3, feature_dim=128)
    config_builder.module(cls=MNISTClassifier, lr=1e-3, feature_dim=128)

    # 2. define a ModelCheckpoint callback
    # checkpoint_callback = ModelCheckpoint(
    #     monitor="val_accuracy", mode="max", save_top_k=3
    # )
    config_builder.checkpointing(monitor="val_accuracy", mode="max", save_top_k=3)

    # 3. Define a Lightning trainer
    # trainer = pl.Trainer(
    #     max_epochs=10,
    #     accelerator="cpu",
    #     strategy="ddp",
    #     log_every_n_steps=100,
    #     logger=CSVLogger("logs"),
    #     callbacks=[checkpoint_callback],
    # )
    config_builder.trainer(
        max_epochs=10,
        accelerator="cpu",
        log_every_n_steps=100,
        logger=CSVLogger("logs"),
    )
    # You do not need to provide the checkpoint callback and strategy here,
    # since LightningTrainer configures them automatically.
    # You can also add any other callbacks into LightningConfigBuilder.trainer().

    # 4. Parameters for model fitting
    # trainer.fit(model, datamodule=datamodule)
    config_builder.fit_params(datamodule=datamodule)

    # Finally, compile all the configs into a dictionary for LightningTrainer
    lightning_config = config_builder.build()
    return lightning_config

Now put everything together:

In [53]:
lightning_config = build_lightning_config_from_existing_code()

In [54]:
scaling_config = ScalingConfig(
    num_workers=4, use_gpu=True, resources_per_worker={"CPU": 1, "GPU": 1}
)

run_config = RunConfig(
    name="ptl-mnist-example",
    storage_path="/tmp/ray_results",
    checkpoint_config=CheckpointConfig(
        num_to_keep=3,
        checkpoint_score_attribute="val_accuracy",
        checkpoint_score_order="max",
    ),
)

In [55]:
if SMOKE_TEST:
    scaling_config = ScalingConfig(
        num_workers=4, use_gpu=False, resources_per_worker={"CPU": 1}
    )

In [56]:
trainer = LightningTrainer(
    lightning_config=lightning_config,
    scaling_config=scaling_config,
    run_config=run_config,
)

Now fit your trainer:

In [57]:
result = trainer.fit()
print("Validation Accuracy: ", result.metrics["val_accuracy"])
result

(pid=358929) /home/ray/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
(pid=358929)   from pandas import MultiIndex, Int64Index
(RayTrainWorker pid=359239) 2023-03-23 17:05:52,362	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=4]
(RayTrainWorker pid=359241) /home/ray/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
(RayTrainWorker pid=359241)   from pandas import MultiIndex, Int64Index
(RayTrainWorker pid=359242) /home/ray/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


Trial name,_report_on,date,done,epoch,experiment_tag,hostname,iterations_since_restore,node_ip,pid,should_checkpoint,step,time_since_restore,time_this_iter_s,time_total_s,timestamp,train_loss,training_iteration,trial_id,val_accuracy,val_loss
LightningTrainer_9cfa6_00000,train_epoch_end,2023-03-23_17-06-20,True,9,0,ip-10-0-61-115,10,10.0.61.115,358929,True,1080,32.1313,2.26905,32.1313,1679616380,0.0822004,10,9cfa6_00000,0.969926,-12.5678


2023-03-23 17:06:23,588	INFO tune.py:817 -- Total run time: 42.87 seconds (42.86 seconds for the tuning loop).


Validation Accuracy:  0.9699258804321289


Result(
  metrics={'_report_on': 'train_epoch_end', 'train_loss': 0.08220043778419495, 'val_accuracy': 0.9699258804321289, 'val_loss': -12.567845344543457, 'epoch': 9, 'step': 1080, 'should_checkpoint': True, 'done': True, 'trial_id': '9cfa6_00000', 'experiment_tag': '0'},
  log_dir=PosixPath('/tmp/ray_results/ptl-mnist-example/LightningTrainer_9cfa6_00000_0_2023-03-23_17-05-40'),
  checkpoint=LightningCheckpoint(local_path=/tmp/ray_results/ptl-mnist-example/LightningTrainer_9cfa6_00000_0_2023-03-23_17-05-40/checkpoint_000009)
)

## Test your network on the test data

Next, we use PyTorch Lightning's native interface to evaluate the best model: To run the test loop using the ``pl.LightningModule.test_step()`` in your user-defined code, simply pass the loaded model to ``pl.Trainer.test()``. 

In [58]:
checkpoint: LightningCheckpoint = result.checkpoint
best_model: pl.LightningModule = checkpoint.get_model(MNISTClassifier)

In [59]:
trainer = pl.Trainer()
test_dataloader = datamodule.test_dataloader()
result = trainer.test(best_model, dataloaders=test_dataloader)

/home/ray/anaconda3/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:92: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9742000102996826     │
└───────────────────────────┴───────────────────────────┘

You can also use {class}`LightningPredictor <ray.train.lightning.LightningPredictor>` for inference.

In [60]:
from ray.train.lightning import LightningPredictor

predictor = LightningPredictor.from_checkpoint(
    checkpoint, MNISTClassifier, use_gpu=False
)


def accuracy(logits, labels):
    preds = np.argmax(logits, axis=1)
    correct_preds = np.sum(preds == labels)
    return correct_preds


corrects = total = 0
for batch in test_dataloader:
    inputs, labels = batch
    inputs, labels = inputs.numpy(), labels.numpy()
    logits = predictor.predict(inputs)["predictions"]
    total += labels.size
    corrects += accuracy(logits, labels)

print("Accuracy: ", corrects / total)

Accuracy:  0.9742


## What's next?
- Use Ray Data for more efficient data preprocessing.
- Use {class}`BatchPredictor <ray.train.batch_prediction.BatchPredictor>` for large-scale distributed inference.
- Find the best hyperparameter settings with Ray Tune.